In [1]:
clear

X=rand(3,100);

X=X./repmat(sqrt(sum(X.*X)),size(X,1),1);

Y=rand(3,1000);

Y=Y./repmat(sqrt(sum(Y.*Y)),size(Y,1),1);

A=X'*Y;

B=X'*X;

I=randi([1,100],1,10);

J=randi([1,1000],1,10);

index=sub2ind([100,1000],I,J);

C=A(index);

A(index)=nan;

In [ ]:
A_filled = A;
nan_pos = isnan(A);
A_filled(nan_pos) = 0;

% 参数设置
max_iter = 100; % 最大迭代次数
tolerance = 1e-5; % 收敛阈值

% 迭代估算缺失值
for iter = 1:max_iter
    A_prev = A_filled; % 保存上一轮的估算结果
    
    % 遍历缺失值
    for i = 1:100
        for j = 1:1000
            if nan_mask(i, j) % 如果是缺失值
                % 使用加权平均估算
                weights = B(i, :); % 用户i与其他用户的相似度
                weights(i) = 0; % 排除自身相似度
                valid = ~nan_mask(:, j); % 有效评分位置
                A_filled(i, j) = sum(weights(valid) .* A_filled(valid, j)) / sum(weights(valid) + eps);
            end
        end
    end
    
    % 检查收敛
    if max(abs(A_filled(:) - A_prev(:))) < tolerance
        disp(['Converged in ', num2str(iter), ' iterations.']);
        break;
    end
end

% 输出结果
disp('原始缺失值：');
disp(C);

disp('估算值：');
disp(A_filled(index));